In [ ]:
import pathlib
import os
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rc
# rc('font',**{'family':'serif','serif':['Helvetica']})
mpl.rcParams['text.usetex'] = True
mpl.rcParams.update({'font.size': 10 * 2.54})
mpl.rcParams['text.latex.preamble']=r"\usepackage{bm}\usepackage{amsmath}"

import jax
import jax.numpy as jnp
import jax_dataclasses as jdc
from jax.tree_util import tree_flatten, tree_unflatten

gpus = jax.devices()
jax.config.update("jax_default_device", gpus[0])

import diffrax

In [ ]:
import exciting_environments as excenvs

from dmpe.utils.density_estimation import select_bandwidth
from dmpe.utils.signals import aprbs
from dmpe.models import NeuralEulerODE
from dmpe.algorithms import excite_with_dmpe
from dmpe.related_work.algorithms import excite_with_sGOATS

from dmpe.evaluation.experiment_utils import (
    get_experiment_ids, load_experiment_results, quick_eval, evaluate_experiment_metrics, evaluate_algorithm_metrics, evaluate_metrics
)

# data generation:

## generate example data with dmpe:

In [ ]:
# seed=2222

# env_params = dict(batch_size=1, tau=5e-1, env_solver=diffrax.Euler())
# env = excenvs.make(
#     "FluidTank-v0",
#     tau=env_params["tau"],
#     solver=env_params["env_solver"]
# )

# alg_params = dict(
#     bandwidth=0.05,
#     n_prediction_steps=100,
#     points_per_dim=50,
#     action_lr=1e-3,
#     n_opt_steps=100,
#     rho_obs=1,
#     rho_act=1,
#     penalty_order=1,
#     clip_action=False,
# )

# model_trainer_params = dict(
#     start_learning=alg_params["n_prediction_steps"],
#     training_batch_size=128,
#     n_train_steps=1,
#     sequence_length=alg_params["n_prediction_steps"],
#     featurize=lambda obs: obs,
#     model_lr=1e-4,
# )
# model_params = dict(obs_dim=env.physical_state_dim, action_dim=env.action_dim, width_size=128, depth=3, key=None)

# exp_params = dict(
#     seed=seed,
#     n_timesteps=5_000,
#     model_class=NeuralEulerODE,
#     env_params=env_params,
#     alg_params=alg_params,
#     model_trainer_params=model_trainer_params,
# )

# key = jax.random.PRNGKey(seed=exp_params["seed"])
# data_key, model_key, loader_key, expl_key, key = jax.random.split(key, 5)

# model_params["key"] = model_key
# exp_params["model_params"] = model_params

# # initial guess
# proposed_actions = aprbs(alg_params["n_prediction_steps"], env.batch_size, 1, 10, data_key)[0]

# # run excitation algorithm
# dmpe_observations, dmpe_actions, model, density_estimate, losses, proposed_actions = excite_with_dmpe(
#     env, exp_params, proposed_actions, loader_key, expl_key, plot_every=100
# )

# np.save("results/qualitative/dmpe_observations.npy", np.stack(dmpe_observations))
# np.save("results/qualitative/dmpe_actions.npy", np.stack(dmpe_actions))

dmpe_observations = np.load("results/qualitative/dmpe_observations.npy")
dmpe_actions = np.load("results/qualitative/dmpe_actions.npy")

In [ ]:
results_path = pathlib.Path("/home/hvater@uni-paderborn.de/projects/exciting-exciting-systems/eval/results/perfect_model_dmpe/fluid_tank")
params, dmpe_observations, dmpe_actions, model = load_experiment_results(get_experiment_ids(results_path)[0], results_path, None)

## generate example data with sGOATS:

In [ ]:
# sgoats_observations, sgoats_actions = excite_with_sGOATS(
#     n_amplitudes=200,
#     n_amplitude_groups=5,
#     reuse_observations=True,
#     env=env,
#     bounds_duration=(1, 50),
#     population_size=50,
#     n_generations=100,
#     featurize=lambda obs: obs,
#     rng=np.random.default_rng(seed=seed),
#     verbose=True
# )

# np.save("results/qualitative/sgoats_observations.npy", np.stack(sgoats_observations))
# np.save("results/qualitative/sgoats_actions.npy", np.stack(sgoats_actions[:-1, :]))

sgoats_observations = np.load("results/qualitative/sgoats_observations.npy")
sgoats_actions = np.load("results/qualitative/sgoats_actions.npy")

In [ ]:
results_path = pathlib.Path("/home/hvater@uni-paderborn.de/projects/exciting-exciting-systems/eval/results/igoats/fluid_tank")
params, igoats_observations, igoats_actions, model = load_experiment_results(get_experiment_ids(results_path)[0], results_path, None)

# qualitative comparison:

In [ ]:
tau = 5
obs_labels=[r"$\tilde{h}$"]
action_labels=[r"$\tilde{q}_{\mathrm{in}}$"]

full_column_width = 18.2
half_colmun_width = 8.89


In [ ]:
def plot_results(observations, actions, tau, obs_labels, action_labels, color, fig=None, axs=None, dotted=False):    
    
    
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(full_column_width, full_column_width/3 * 0.7))
     
    t = jnp.linspace(0, observations.shape[0] - 1, observations.shape[0]) * tau
    
    for observation_idx in range(observations.shape[-1]):
        axs[0].plot(
            t,
            jnp.squeeze(observations[..., observation_idx]),
            "." if dotted else "-",
            markersize=1,
            label=obs_labels[observation_idx],
            color=c,
        )
    #axs[0].title.set_text("observations, timeseries")
    # axs[0].legend(loc='lower right')
    axs[0].set_ylabel(r"$\tilde{x}$", labelpad=-15)
    axs[0].set_xlabel("$t \mathrm{\ in \ } s$")


    if observations.shape[0] == actions.shape[0] + 1:
        observations_ = observations[:-1]
        t = t[:-1]
    else:
        observations_ = observations
    
    axs[1].scatter(jnp.squeeze(actions[..., 0]), jnp.squeeze(observations_[..., 0]), s=1, color=c,)
    #axs[1].title.set_text("observation $\\times$ action plane")
    axs[1].set_ylabel(obs_labels[0], labelpad=-15)
    axs[1].set_xlabel(action_labels[0])
    
    for action_idx in range(actions.shape[-1]):
        axs[2].plot(t, jnp.squeeze(actions[..., action_idx]), label=action_labels[action_idx], color=c,)
    
    # axs[2].legend(loc='lower right')
    axs[2].set_ylabel(r"$\tilde{u}$", labelpad=-15)
    axs[2].set_xlabel("$t \mathrm{\ in \ } s$")
    
    for ax in axs:
        ax.grid(True)
    fig.tight_layout(pad=0.3)
    
    [ax.tick_params(axis="y", direction='in') for ax in axs]
    [ax.tick_params(axis="x", direction='in') for ax in axs]
    
    return fig, axs

In [ ]:
colors = plt.rcParams["axes.prop_cycle"]()
c = next(colors)["color"]
c = next(colors)["color"]

fig, axs = plot_results(
    observations=dmpe_observations[:1000],
    actions=dmpe_actions[:1000],
    tau=tau,
    obs_labels=obs_labels,
    action_labels=action_labels,
    color=c,
);

plt.savefig("results/qualitative/qualitative_results_fluidtank_dmpe.pdf")
plt.show()

In [ ]:
c = next(colors)["color"]
c = next(colors)["color"]

if c == '#d62728':
    c = next(colors)["color"]

fig, axs = plot_results(
    observations=igoats_observations[:1000],
    actions=igoats_actions[:1000],
    tau=tau,
    obs_labels=obs_labels,
    action_labels=action_labels,
    color=c
);
plt.savefig("results/qualitative/qualitative_results_fluidtank_igoats.pdf")
plt.show()

---

In [ ]:
from dmpe.utils.density_estimation import DensityEstimate

In [ ]:
select_bandwidth(2, 2, 100, 0.5)

In [ ]:
dmpe_density_estimate = DensityEstimate.from_dataset(dmpe_observations, dmpe_actions, bandwidth=select_bandwidth(2, 2, 50, 0.3), points_per_dim=50)
igoats_density_estimate = DensityEstimate.from_dataset(igoats_observations, igoats_actions, bandwidth=select_bandwidth(2, 2, 50, 0.3), points_per_dim=50)

In [ ]:
def plot_2d_kde_as_contourf(p_est, x, observation_label, action_label):

    fig, axs = plt.subplots(figsize=(half_colmun_width, half_colmun_width * 8/9*0.7))

    grid_len_per_dim = int(np.sqrt(x.shape[0]))
    x_plot = x.reshape((grid_len_per_dim, grid_len_per_dim, 2))

    cax = axs.contourf(
        x_plot[..., 1],
        x_plot[..., 0],
        p_est.reshape(x_plot.shape[:-1]),
        antialiased=False,
        levels=np.linspace(0., 1.5, 31),
        alpha=1.0,
        cmap=plt.cm.coolwarm,
    )
    axs.set_xlabel(action_label, labelpad=0)
    axs.set_ylabel(observation_label, labelpad=-15)

    # axs.tick_params(axis="y", direction='in')
    # axs.tick_params(axis="x", direction='in')

    plt.tight_layout(pad=0.25)
    cbar = fig.colorbar(cax)
    return fig, axs, cax


In [ ]:
np.linspace(0., 1.5, 31)

In [ ]:
fig, axs, cax = plot_2d_kde_as_contourf(igoats_density_estimate.p, igoats_density_estimate.x_g, obs_labels[0], action_labels[0])
mpl.rcParams.update({'figure.autolayout': True})
plt.savefig("results/qualitative/qualitative_kde_fluidtank_igoats.pdf")
plt.show()

In [ ]:
fig, axs, cax = plot_2d_kde_as_contourf(dmpe_density_estimate.p, dmpe_density_estimate.x_g, obs_labels[0], action_labels[0])
mpl.rcParams.update({'figure.autolayout': True})
plt.savefig("results/qualitative/qualitative_kde_fluidtank_dmpe.pdf")
plt.show()